In [1]:
import pandas as pd
import sys
sys.path.append('..')
sys.path.append('../..')
sys.path.append('../../..')

import os
from dotenv import load_dotenv
load_dotenv()

import ssl
ssl._create_default_https_context = ssl._create_stdlib_context

from helpers.focusmate import fm_sessions_data_to_df
from api.helpers.focusmate import fetch_all_focusmate_sessions, fetch_focusmate_profile
from api.helpers.request import get_access_token_from_db

fm_api_profile_endpoint = os.getenv("NEXT_PUBLIC_FM_API_PROFILE_ENDPOINT")
fm_api_sessions_endpoint = os.getenv("NEXT_PUBLIC_FM_API_SESSIONS_ENDPOINT")
session_id = ""
local_timezone = "America/New_York"

In [2]:
access_token = get_access_token_from_db(session_id)

profile_data = fetch_focusmate_profile(
        fm_api_profile_endpoint, access_token).get("user")
local_timezone: str = profile_data.get("timeZone")
member_since: str = profile_data.get("memberSince")

sessions_data = await fetch_all_focusmate_sessions(
    fm_api_sessions_endpoint, access_token, member_since)

all_sessions = fm_sessions_data_to_df(sessions_data, local_timezone)
sessions = all_sessions[all_sessions['completed'] == True].copy()

In [3]:
sessions['start_time_month'] = sessions['start_time'].dt.to_period('M')

# Get the last month
last_month = (pd.Timestamp.today().to_period('M') - 1)

# Create a complete range of months up to the last month
full_month_range = pd.period_range(start=sessions['start_time_month'].min(), end=last_month, freq='M')

# Convert months to a set for quick lookup
months_set = set(sessions['start_time_month'])

# Initialize the streak counter
monthly_streak = 0

# Check for consecutive months in the range
for month in reversed(full_month_range):
    if month in months_set:
        monthly_streak += 1
    else:
        break

In [4]:
print(last_month)
print(full_month_range)
print(months_set)
print(monthly_streak)

2024-05
PeriodIndex(['2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09',
             '2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-03',
             '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
             '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
             '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09',
             '2023-10', '2023-11', '2023-12', '2024-01', '2024-02', '2024-03',
             '2024-04', '2024-05'],
            dtype='period[M]')
{Period('2021-10', 'M'), Period('2022-09', 'M'), Period('2023-08', 'M'), Period('2023-10', 'M'), Period('2023-01', 'M'), Period('2022-11', 'M'), Period('2022-02', 'M'), Period('2023-12', 'M'), Period('2021-05', 'M'), Period('2023-03', 'M'), Period('2024-02', 'M'), Period('2023-05', 'M'), Period('2022-04', 'M'), Period('2021-07', 'M'), Period('2022-06', 'M'), Period('2024-04', 'M'), Period('2021-09', 'M'), Period('2022-08', 'M'), Period('2023-07', 